In [27]:
# bsd `date` cannot print milliseconds. sigh
t1=$(($(gdate +%s%N)/1000000))

for i in $(cat input.txt); do
  for j in $(cat input.txt); do
    if [[ $((i+j)) == 2020 ]]; then
      echo $i, $j, $((i+j)), $((i * j))
      break 2
    fi
  done
done

t2=$(($(gdate +%s%N)/1000000))
echo "took $((t2-t1))ms"

1486, 534, 2020, 793524
took 715ms


In [29]:
t1=$(($(gdate +%s%N)/1000000))

# this time, let's save the length of the file and only sum up the numbers past where we are
n=$(wc -l < input.txt)
for i in $(cat input.txt); do
  n=$((n-1))
  for j in $(cat input.txt | tail -n $n); do
    if [[ $((i+j)) == 2020 ]]; then
      echo $i, $j, $((i+j)), $((i * j))
      break 2
    fi
  done
done

t2=$(($(gdate +%s%N)/1000000))
echo "took $((t2-t1))ms"

# the tail actually seems to make this take longer!!?!

1486, 534, 2020, 793524
took 884ms


In [25]:
t1=$(($(gdate +%s%N)/1000000))

ni=$(wc -l < input.txt)
for i in $(cat input.txt); do
  ni=$((ni-1))
  nj=ni
  for j in $(cat input.txt | tail -n $ni); do
    nj=$((nj-1))
    if [[ $((i+j)) -gt 2020 ]]; then
      continue
    fi
    for k in $(cat input.txt | tail -n $nj); do
      if [[ $((i+j+k)) == 2020 ]]; then
        echo $i, $j, $k, $((i+j+k)), $((i*j*k))
        break 3
      fi
    done
  done
done

t2=$(($(gdate +%s%N)/1000000))
echo "took $((t2-t1))ms"

686, 71, 1263, 2020, 61515678
took 3434ms


In [26]:
t1=$(($(gdate +%s%N)/1000000))

for i in $(cat input.txt); do
  for j in $(cat input.txt); do
    if [[ $((i+j)) -gt 2020 ]]; then
      continue
    fi
    for k in $(cat input.txt); do
      if [[ $((i+j+k)) == 2020 ]]; then
        echo $i, $j, $k, $((i+j+k)), $((i*j*k))
        break 3
      fi
    done
  done
done

t2=$(($(gdate +%s%N)/1000000))
echo "took $((t2-t1))ms"

# in the triple loop, the tail still doesn't save time

686, 71, 1263, 2020, 61515678
took 3457ms


In [40]:
t1=$(($(gdate +%s%N)/1000000))

# this time let's use proper arrays, and see how much faster it is

# read the inputs.txt file into an array called inputs
# `readarray` is a synonym for `mapfile`. manual here:
# http://www.gnu.org/software/bash/manual/html_node/Bash-Builtins.html#index-mapfile
readarray -t inputs < input.txt

for i in ${inputs[@]}; do
for j in ${inputs[@]}; do
    if [[ $((i+j)) == 2020 ]]; then
      echo $i, $j, $((i*j))
      break 2
    fi
done
done

t2=$(($(gdate +%s%N)/1000000))
echo "took $((t2-t1))ms"

1486, 534, 793524
took 951ms


In [41]:
t1=$(($(gdate +%s%N)/1000000))

readarray -t inputs < input.txt

for i in ${inputs[@]}; do
for j in ${inputs[@]}; do
    if [[ $((i+j)) -gt 2020 ]]; then
      continue
    fi
    for k in ${inputs[@]}; do
        if [[ $((i+j+k)) == 2020 ]]; then
          echo $i, $j, $k, $((i*j*k))
          break 3
        fi
    done
done
done

t2=$(($(gdate +%s%N)/1000000))
echo "took $((t2-t1))ms"

686, 71, 1263, 61515678
took 2224ms


In [44]:
t1=$(($(gdate +%s%N)/1000000))

# we can get a pretty decent speedup by slicing the arrays properly to avoid extra loops

readarray -t inputs < input.txt

ni=0

for i in ${inputs[@]}; do
    ni=$((ni+1))
    nj=ni
    # we can even slice arrays in bash!
    for j in ${inputs[@]:ni}; do
        nj=$((nj+1))
        if [[ $((i+j)) -gt 2020 ]]; then
          continue
        fi
        for k in ${inputs[@]:nj}; do
            if [[ $((i+j+k)) == 2020 ]]; then
              echo $i, $j, $k, $((i*j*k))
              break 3
            fi
        done
    done
done

t2=$(($(gdate +%s%N)/1000000))
echo "took $((t2-t1))ms"

686, 71, 1263, 61515678
took 1819ms
